In order to get some datasets I needed to export them by convertind the dataframe to a csv and then outputting the contents of the file to the terminal such as the example below.
```Python
# output the contents of the dataframe to CSV wile also removing the index for pushing it to a dataframe.
wards_altered.to_csv("wards_altered.csv", index=False)

# open the file and read the contents of the file to the terminal.
with open("wards_altered.csv", "r") as f:
    print(f.read())
```

# Uniformity